### PyTorch Implementation:

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

In [3]:
# Define the CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.fc = nn.Linear(32 * 28 * 28, 10)

    def forward(self, x):
        x = self.relu1(self.conv1(x))
        x = self.relu2(self.conv2(x))
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [4]:
# Define the transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

In [5]:
# Load the MNIST dataset
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

100%|██████████| 9912422/9912422 [00:00<00:00, 160152353.98it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 54247959.62it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 45630473.11it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 21947613.79it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [6]:
# Create the data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

In [7]:
# Create the model
model = CNN()

In [8]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [10]:
# Training loop
epochs = 10
for epoch in range(epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        images = images
        labels = labels

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

In [12]:
# Calculate validation accuracy
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images
        labels = labels
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print('Epoch [{}/{}], Loss: {:.4f}, Validation Accuracy: {:.2f}%'.format(epoch+1, epochs, running_loss, accuracy))

Epoch [10/10], Loss: 3.2076, Validation Accuracy: 98.61%


### TensorFlow Implementation:

In [17]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.datasets import mnist

In [18]:
# Load the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [19]:
# Normalize pixel values between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

In [20]:
# Add a channel dimension to the images
train_images = train_images[..., tf.newaxis]
test_images = test_images[..., tf.newaxis]

In [21]:
# Create the CNN model
model = models.Sequential()
model.add(layers.Conv2D(16, kernel_size=3, strides=1, padding='same', activation='relu', input_shape=(28, 28, 1)))
model.add(layers.Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(10, activation='softmax'))

In [22]:
# Print the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 16)        160       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        4640      
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 10)                250890    
                                                                 
Total params: 255,690
Trainable params: 255,690
Non-trainable params: 0
_________________________________________________________________


In [23]:
# Set the optimizer and loss function
optimizer = optimizers.Adam()
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

In [24]:
# Compile the model
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

In [25]:
# Training loop
epochs = 10
for epoch in range(epochs):
    # Train the model
    history = model.fit(train_images, train_labels, batch_size=64, epochs=1, verbose=1)

938/938 [==============================] - 4s 4ms/step - loss: 0.0055 - accuracy: 0.9980


In [26]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_images, test_labels, verbose=0)

In [27]:
print('Epoch [{}/{}], Loss: {:.4f}, Validation Accuracy: {:.2f}%'.format(epoch+1, epochs, history.history['loss'][0], test_accuracy * 100))

Epoch [10/10], Loss: 0.0055, Validation Accuracy: 98.61%
